In [1]:
import os
import gc
import json
import time
from datetime import datetime
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

#from custom_functions import process_dfs, preprocess

In [2]:
def process_dfs(train_df, test_df):
    print("Processing dfs...")
    print("Dropping repeated columns...")
    columns = [col for col in train_df.columns if train_df[col].nunique() > 1]

    train_df = train_df[columns]
    test_df = test_df[columns]

    trn_len = train_df.shape[0]

    merged_df = pd.concat([train_df, test_df])

    merged_df['diff_visitId_time'] = merged_df['visitId'] - merged_df['visitStartTime']
    merged_df['diff_visitId_time'] = (merged_df['diff_visitId_time'] != 0).astype(int)
    del merged_df['visitId']

    print("Generating date columns...")
    format_str = '%Y%m%d'
    merged_df['formated_date'] = merged_df['date'].apply(lambda x: datetime.strptime(str(x), format_str))
    merged_df['WoY'] = merged_df['formated_date'].apply(lambda x: x.isocalendar()[1])
    merged_df['month'] = merged_df['formated_date'].apply(lambda x:x.month)
    merged_df['quarter_month'] = merged_df['formated_date'].apply(lambda x:x.day//8)
    merged_df['weekday'] = merged_df['formated_date'].apply(lambda x:x.weekday())

    del merged_df['date']
    del merged_df['formated_date']

    merged_df['formated_visitStartTime'] = merged_df['visitStartTime'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x)))
    merged_df['formated_visitStartTime'] = pd.to_datetime(merged_df['formated_visitStartTime'])
    merged_df['visit_hour'] = merged_df['formated_visitStartTime'].apply(lambda x: x.hour)

    del merged_df['visitStartTime']
    del merged_df['formated_visitStartTime']

    print("Encoding columns with pd.factorize()")
    for col in merged_df.columns:
        if col in ['fullVisitorId', 'month', 'quarter_month', 'weekday', 'visit_hour', 'WoY']: continue
        if merged_df[col].dtypes == object or merged_df[col].dtypes == bool: merged_df[col], indexer = pd.factorize(merged_df[col])

    print("Splitting back...")
    train_df = merged_df[:trn_len]
    test_df = merged_df[trn_len:]
    print("Done!")

    return train_df, test_df

In [3]:
def preprocess():
    train_df = pd.read_csv('train-flattened.csv', dtype = {'fullVisitorId' : np.str})
    test_df = pd.read_csv('test-flattened.csv', dtype = {'fullVisitorId' : np.str})

    target = train_df['totals.transactionRevenue'].fillna(0).astype(float)
    target = target.apply(lambda x: np.log1p(x))

    del train_df['totals.transactionRevenue']

    train_df, test_df = process_dfs(train_df, test_df)
    train_df.to_csv('train-flat-clean.csv', index=False)
    test_df.to_csv('test-flat-clean.csv', index=False)
    target.to_csv('target.csv', index=False)

In [4]:
def rmse(y_true, y_pred):
    return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)

def load_preprocessed_dfs(drop_full_visitor_id=True):

    X_train = pd.read_csv('train-flat-clean.csv', converters={'fullVisitorId': str})
    X_test = pd.read_csv('test-flat-clean.csv', converters={'fullVisitorId': str})
    y_train = pd.read_csv('target.csv', names=['LogRevenue']).T.squeeze()
    
    # This is the only `object` column, we drop it for train and evaluation
    if drop_full_visitor_id: 
        X_train = X_train.drop(['fullVisitorId'], axis=1)
        X_test = X_test.drop(['fullVisitorId'], axis=1)
    return X_train, y_train, X_test

In [5]:
preprocess()

Processing dfs...
Dropping repeated columns...
Generating date columns...
Encoding columns with pd.factorize()
Splitting back...
Done!


In [6]:
X, y, X_test = load_preprocessed_dfs()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=1)

print(f"Train shape: {X_train.shape}")
print(f"Validation shape: {X_val.shape}")
print(f"Test (submit) shape: {X_test.shape}")

Train shape: (1452086, 28)
Validation shape: (256251, 28)
Test (submit) shape: (401589, 28)


In [13]:
def run_catboost(X_train, y_train, X_val, y_val, X_test):
    model = CatBoostRegressor(iterations=1000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50, 
                             od_wait=20)
    model.fit(X_train, y_train,
              eval_set=(X_val, y_val),
              use_best_model=True,
              verbose=50)
    
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    y_pred_submit = model.predict(X_test)

    print(f"CatB: RMSE val: {rmse(y_val, y_pred_val)}  - RMSE train: {rmse(y_train, y_pred_train)}")
    return y_pred_submit, model

In [14]:
%%time
# Train Catboost and generate predictions
cat_preds, cat_model = run_catboost(X_train, y_train, X_val, y_val,  X_test)

0:	learn: 1.8293702	test: 1.8260435	best: 1.8260435 (0)	total: 1.29s	remaining: 21m 31s
50:	learn: 1.5428422	test: 1.5470487	best: 1.5470487 (50)	total: 56.9s	remaining: 17m 38s
100:	learn: 1.5191854	test: 1.5337330	best: 1.5337330 (100)	total: 1m 52s	remaining: 16m 43s
150:	learn: 1.5034843	test: 1.5286896	best: 1.5286896 (150)	total: 2m 48s	remaining: 15m 45s
200:	learn: 1.4902613	test: 1.5253440	best: 1.5253438 (199)	total: 3m 43s	remaining: 14m 48s
250:	learn: 1.4780556	test: 1.5228015	best: 1.5228015 (250)	total: 4m 39s	remaining: 13m 52s
300:	learn: 1.4673304	test: 1.5217889	best: 1.5217889 (300)	total: 5m 33s	remaining: 12m 54s
350:	learn: 1.4583313	test: 1.5206993	best: 1.5206233 (341)	total: 6m 28s	remaining: 11m 58s
400:	learn: 1.4487933	test: 1.5193488	best: 1.5193447 (398)	total: 7m 23s	remaining: 11m 2s
450:	learn: 1.4393074	test: 1.5186329	best: 1.5184995 (443)	total: 8m 18s	remaining: 10m 6s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 1.518499525
be